# Week 5 Exercise 

In [1]:
import graphlab as gl
gl.canvas.set_target('ipynb')

In [2]:
song_data = gl.SFrame('Data/song_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to huangchengyu16@gmail.com and will expire on September 03, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1473775573.log


In [7]:
## print all variable names ##
song_data.column_names()

['user_id', 'song_id', 'listen_count', 'title', 'artist', 'song']

### Q1. Look at unique users for different artist

In [14]:
## creating a list of unique users for a couple of songs ###
west = song_data[song_data['artist']=='Kanye West']
fighters = song_data[song_data['artist']=='Foo Fighters']
swift = song_data[song_data['artist']=='Taylor Swift']
gaga = song_data[song_data['artist']=='Lady GaGa']

users = song_data['user_id'].unique()

#### Look at artilist with highest unique users

In [49]:
artilist_list = [west,fighters,swift,gaga]
artilist_user_list = [(x['artist'][0],len(x['user_id'].unique())) for x in artilist_list]

In [48]:
artilist_names,unique_user = map(list,zip(*artilist_user_list))
rank_list = gl.SFrame({'names':artilist_names,'users':unique_user})
rank_list.sort('users',ascending=False)

names,users
Taylor Swift,3246
Lady GaGa,2928
Kanye West,2522
Foo Fighters,2055


### Q2 . User group by to find the most popular and least popular artist

In [22]:
artist_listen_count = song_data.groupby(key_columns='artist', operations={'total_count': gl.aggregate.SUM('listen_count')})
artist_listen_count.sort('total_count',ascending=False)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [51]:
artist_listen_count.sort('total_count')

artist,total_count
William Tabbert,14
Reel Feelings,24
Beyoncé feat. Bun B andSlim Thug ...,26
Boggle Karaoke,30
Diplo,30
harvey summers,31
Nâdiya,36
Jody Bernal,38
Aneta Langerova,38
Kanye West / Talib Kweli/ Q-Tip / Common / ...,38


### Q3. Using groupby-aggregate to find the most recommended songs

In [23]:
## split training and test data
train_data,test_data = song_data.random_split(.8,seed=0)

In [24]:
## using item similarity model 
personalized_model = gl.item_similarity_recommender.create(train_data,user_id='user_id',item_id='song')

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 3.14s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 4.35455s

In [25]:
## only select a subset of all the users to same some time 
subset_test_users = test_data['user_id'].unique()[0:10000]

In [27]:
## recommend 1 song for each of these test users 
rec_list = personalized_model.recommend(subset_test_users,k=1)

recommendations finished on 1000/10000 queries. users per second: 14589.4

recommendations finished on 2000/10000 queries. users per second: 7957.19

recommendations finished on 3000/10000 queries. users per second: 10530.8

recommendations finished on 4000/10000 queries. users per second: 8704.03

recommendations finished on 5000/10000 queries. users per second: 7786.2

recommendations finished on 6000/10000 queries. users per second: 8916.07

recommendations finished on 7000/10000 queries. users per second: 8229.29

recommendations finished on 8000/10000 queries. users per second: 9091.4

recommendations finished on 9000/10000 queries. users per second: 8477.98

recommendations finished on 10000/10000 queries. users per second: 8955.36

In [28]:
rec_list.head()

user_id,song,score,rank
c067c22072a17d33310d7223d7b79f819e48cf42 ...,Grind With Me (ExplicitVersion) - Pretty Ricky ...,0.0459424376488,1
696787172dd3f5169dc94deef97e427cee86147d ...,Senza Una Donna (WithoutA Woman) - Zucchero / ...,0.017026577677,1
532e98155cbfd1e1a474a28ed96e59e50f7c5baf ...,Jive Talkin' (AlbumVersion) - Bee Gees ...,0.0118288653237,1
18325842a941bc58449ee71d659a08d1c1bd2383 ...,Goodnight And Goodbye -Jonas Brothers ...,0.0159257985651,1
507433946f534f5d25ad1be302edb9a2376f503c ...,Find The Cost Of Freedom- Crosby_ Stills_ Nash & ...,0.0165806589303,1
18fafad477f9d72ff86f7d0bd838a6573de0f64a ...,Rabbit Heart (Raise ItUp) - Florence + The ...,0.0799399726093,1
fe85b96ba1983219b296f6b4869dd29eb2b72ff9 ...,Secrets - OneRepublic,0.0788827141126,1
225ea420b4bede50919d1bfe24a599691522d176 ...,Clocks - Coldplay,0.0271030251796,1
95dc7e2b188b1148b2d25f4e6b6e94afacc4efc3 ...,Bust a Move - InfectedMushroom ...,0.0534738540649,1
4a3a1ae2748f12f7ab921a47d6d79abf82e3e325 ...,Isis (Spam Remix) -Alaska Y Dinarama ...,0.04180302118,1


In [31]:
agg_rec_list = rec_list.groupby(key_columns='song', operations={'total_count': gl.aggregate.COUNT()})
agg_rec_list.sort('total_count',ascending=False)

song,total_count
Undo - Björk,429
Secrets - OneRepublic,373
Revelry - Kings Of Leon,222
You're The One - DwightYoakam ...,161
Fireflies - CharttraxxKaraoke ...,111
Sehr kosmisch - Harmonia,106
Hey_ Soul Sister - Train,100
Horn Concerto No. 4 in Eflat K495: II. Romance ...,89
OMG - Usher featuringwill.i.am ...,60
The Scientist - Coldplay,43
